In [55]:
import os
from PIL import Image
import numpy as np

# Generate

In [56]:
PATH_TO_TILES = "Tiles"
PATH_TO_COORDS = "Coordinates"
GENERATED_DIR = "Generated"
IMAGE_SIZE_W = 256
IMAGE_SIZE_L = 256

os.makedirs(GENERATED_DIR, exist_ok=True)

for subfolder in os.listdir(PATH_TO_COORDS):
    coords_subfolder_path = os.path.join(PATH_TO_COORDS, subfolder)
    tiles_subfolder_path = os.path.join(PATH_TO_TILES, subfolder)
    generated_subfolder_path = os.path.join(GENERATED_DIR, subfolder)

    if not os.path.exists(tiles_subfolder_path):
        continue

    os.makedirs(generated_subfolder_path, exist_ok=True)

    for coord_file in os.listdir(coords_subfolder_path):
        if coord_file.endswith("_coords.txt"):
            base_name = coord_file[:-11]

            coord_file_path = os.path.join(coords_subfolder_path, coord_file)
            tile_image_path = os.path.join(tiles_subfolder_path, base_name + ".png")

            if not os.path.exists(tile_image_path):
                continue

            with open(coord_file_path, 'r') as file:
                line_count = file.read().count('\n')

            label = "low" if line_count < 5 else "high"

            with Image.open(tile_image_path) as img:
                resized_img = img.resize((IMAGE_SIZE_W, IMAGE_SIZE_L))
                output_file_name = f"{base_name}_{label}.png"
                output_file_path = os.path.join(generated_subfolder_path, output_file_name)
                resized_img.save(output_file_path)

# Loading

In [64]:
image_data = np.empty((0, 256, 256, 3), dtype=np.float32) 
labels = np.empty((0,), dtype=np.int32)

def label_to_numeric(label):
    return 0 if label == "low" else 1  # Map "low" to 0 and "high" to 1

for subfolder in os.listdir(GENERATED_DIR):
    subfolder_path = os.path.join(GENERATED_DIR, subfolder)

    if os.path.isdir(subfolder_path): 
        for file_name in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file_name)

            if file_name.endswith(".png"):
                label = file_name.split("_")[-1].split(".")[0]  # Get "low" or "high"

                with Image.open(file_path) as img:
                    if img.mode == "RGBA":
                        img = img.convert("RGB")

                    img_array = np.array(img) / 255.0  

                    if img_array.shape != (256, 256, 3):
                        print(f"Skipping image due to unexpected shape: {img_array.shape}")
                        continue

                    image_data = np.vstack([image_data, img_array[np.newaxis, ...]])
                    labels = np.append(labels, label_to_numeric(label))



# Pytorch

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class GeneratedDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = torch.tensor(self.images[idx], dtype=torch.float32).permute(2, 0, 1)  
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return image, label

dataset = GeneratedDataset(image_data, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch_images, batch_labels in dataloader:
    print(batch_images.shape, batch_labels.shape)
    break

# Tensorflow

In [74]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices((image_data, labels))

dataset = dataset.shuffle(len(image_data)).batch(32)
